In [2]:
import pandas as pd

In [2]:
df = pd.read_csv('all_classes.csv', index_col=False)
df.drop('ID', axis=1,inplace=True)
df.index = df.index+1
df.head()

,Student,Period,Gender,Class,Subject,Mark,Is_new_sub,Average_grade,Perform_trend
1,1,1,М,8,Алг.,3,1,3.89,0
2,1,2,М,8,Алг.,3,0,3.89,0
3,1,3,М,8,Алг.,3,0,3.89,0
4,1,4,М,8,Алг.,3,0,3.96,0
5,2,1,М,8,Алг.,3,1,3.39,0


In [3]:
categorical_columns = df.loc[:, df.dtypes == object].columns
### mean encoding'а

def encode_multiclass_target(df, category, target='Mark'):
    df = df[[category, target]].copy()
    
    target_dummies = pd.get_dummies(df[target], 
                                    prefix=category, 
                                    drop_first=True)
    
    df = pd.concat((df, target_dummies), axis=1)
    
    for tg in target_dummies.columns:
        
        df[tg] = df.groupby(category)[tg].transform("mean")
    
    return df.drop(category, axis=1)
one_hot = pd.get_dummies(df['Gender'], prefix='gender', drop_first=True)
df1 = pd.concat([df, one_hot], axis=1)
upd_df = df.copy()
### Производим encoding
for col in categorical_columns:
    if upd_df[col].nunique() < 4:
        one_hot = pd.get_dummies(upd_df[col], prefix=col, drop_first=True)     
        upd_df = pd.concat((upd_df, one_hot), axis=1)
    else:
        mean_target = encode_multiclass_target(upd_df, col).drop('Mark', axis=1)
        upd_df = pd.concat((upd_df.drop(col, axis=1), mean_target), axis=1)
upd_df = upd_df.drop('Gender', axis=1)

In [4]:
import joblib
model = joblib.load('model_1.pkl')

In [5]:
y = model.predict(upd_df.drop('Mark', axis=1))

In [6]:
y = pd.DataFrame(y)

In [7]:
df  = df.join(y)

In [8]:
df.columns = ['Student',        'Period',        'Gender',         'Class',
             'Subject',          'Mark',    'Is_new_sub', 'Average_grade',
       'Perform_trend', 'y']

In [9]:
df.drop('Mark', axis = 1,inplace=True)

In [10]:
df.loc[5019, 'y'] = 5

In [11]:
df

,Student,Period,Gender,Class,Subject,Is_new_sub,Average_grade,Perform_trend,y
1,1,1,М,8,Алг.,1,3.89,0,3.0
2,1,2,М,8,Алг.,0,3.89,0,3.0
3,1,3,М,8,Алг.,0,3.89,0,3.0
4,1,4,М,8,Алг.,0,3.96,0,3.0
5,2,1,М,8,Алг.,1,3.39,0,3.0
...,...,...,...,...,...,...,...,...,...
5015,72,2,Ж,10,Химия,0,4.38,1,4.0
5016,72,1,Ж,11,Химия,0,4.38,-1,5.0
5017,73,1,М,10,Химия,1,4.94,0,5.0
5018,73,2,М,10,Химия,0,4.94,0,5.0


In [12]:
student_index = df.Student.unique()

In [13]:
import pandas as pd
import random
random.seed(0)
# Примеры мужских и женских имен, фамилий и отчеств
male_names = ['Алексей', 'Иван', 'Дмитрий', 'Сергей', 'Михаил', 'Павел', 'Егор', 'Николай', 'Виктор', 'Андрей']
female_names = ['Екатерина', 'Анна', 'Мария', 'Ольга', 'Наталья', 'Елена', 'Ирина', 'Татьяна', 'Светлана', 'Любовь']

male_surnames = ['Иванов', 'Петров', 'Сидоров', 'Кузнецов', 'Попов', 'Смирнов', 'Васильев', 'Морозов', 'Новиков', 'Фёдоров']
female_surnames = [s + 'а' for s in male_surnames]

male_patronymics = ['Александрович', 'Иванович', 'Дмитриевич', 'Сергеевич', 'Михайлович', 'Павлович', 'Егорович', 'Николаевич', 'Викторович', 'Андреевич']
female_patronymics = ['Александровна', 'Ивановна', 'Дмитриевна', 'Сергеевна', 'Михайловна', 'Павловна', 'Егоровна', 'Николаевна', 'Викторовна', 'Андреевна']

# Определим количество мужчин и женщин
num_total = 73
num_male = random.randint(30, 43)  # случайное число мужчин
num_female = num_total - num_male

# Генерация мужских ФИО
male_data = [
    {
        "surname": random.choice(male_surnames),
        "name": random.choice(male_names),
        "patronymic": random.choice(male_patronymics)
    }
    for _ in range(num_male)
]

# Генерация женских ФИО
female_data = [
    {
        "surname": random.choice(female_surnames),
        "name": random.choice(female_names),
        "patronymic": random.choice(female_patronymics)
    }
    for _ in range(num_female)
]

# Объединяем данные
full_data = male_data + female_data
random.shuffle(full_data)  # перемешиваем

# Создаем DataFrame
fio = pd.DataFrame(full_data)



In [14]:
student_info = fio.set_index(student_index)
student_info

,surname,name,patronymic
1,Сидоров,Михаил,Иванович
2,Фёдоров,Николай,Павлович
26,Сидоров,Иван,Андреевич
3,Попов,Иван,Викторович
4,Новиков,Иван,Павлович
...,...,...,...
69,Васильев,Павел,Андреевич
70,Фёдорова,Анна,Михайловна
71,Иванова,Любовь,Александровна
72,Смирнов,Сергей,Сергеевич


In [15]:
columns = ['number', 'letter']
classes = pd.DataFrame(
    [[8, 'А'], [8, 'Б'], [8,'В'], [9,'А'], [9,'Б'], [10,'А'], [11,'А']
], columns=columns)


In [16]:
classes['class_id'] = classes.index
classes = classes[['class_id', 'number', 'letter']]

In [17]:
classes

,class_id,number,letter
0,0,8,А
1,1,8,Б
2,2,8,В
3,3,9,А
4,4,9,Б
5,5,10,А
6,6,11,А


In [18]:
temp_table = pd.DataFrame(student_index).join(df['Class'])
temp_table

,0,Class
0,1,NaN
1,2,8.0
2,26,8.0
3,3,8.0
4,4,8.0
...,...,...
68,69,8.0
69,70,8.0
70,71,8.0
71,72,8.0


In [19]:
student_info = student_info.join(temp_table['Class'])

In [20]:
student_info.loc[73, 'Class'] = 10

In [21]:
class_code = [0,1,2,3,6,4,3,4,1,4,3,5,6,5,2,0,1,3,4,5,0,2,6,3,0,5,1,2,4,1,5,6,6,0,5,5,5,5,6,5,2,1,4,6,2,6,5,5,0,3,0,5,6,1,3,6,2,6,0,4,1,3,6,6,6,2,4,1,4,5,6,3,2]
student_info['class_id'] = class_code

In [22]:
student_info.drop('Class', inplace=True, axis =1)

In [23]:
student_info['student_id'] = student_info.index
student_info = student_info[['student_id','surname',	'name',	'patronymic',	'class_id']]

In [24]:
student_info.sort_values('student_id', inplace=True)

In [25]:
student_info

,student_id,surname,name,patronymic,class_id
1,1,Сидоров,Михаил,Иванович,0
2,2,Фёдоров,Николай,Павлович,1
3,3,Попов,Иван,Викторович,3
4,4,Новиков,Иван,Павлович,6
5,5,Петров,Михаил,Викторович,4
...,...,...,...,...,...
69,69,Васильев,Павел,Андреевич,4
70,70,Фёдорова,Анна,Михайловна,5
71,71,Иванова,Любовь,Александровна,6
72,72,Смирнов,Сергей,Сергеевич,3


In [26]:
subjects = pd.DataFrame(df['Subject'].unique())
subjects['id'] = subjects.index
subjects = subjects[['id', 0]]
subjects.columns = ['id', 'Subject_name']
subjects.head()

,id,Subject_name
0,0,Алг.
1,1,Англ.яз
2,2,Биол.
3,3,ГД
4,4,Геогр.


In [27]:
students = df[['Student','Subject','y']].copy()
students.columns = ['student_id', 'Subject_name', 'y']
students = students.join(subjects.set_index('Subject_name'), on='Subject_name')
students.drop('Subject_name', axis = 1, inplace=True)
students.columns = ['student_id', 'grade', 'subject_id']
students = students[['student_id', 'subject_id', 'grade']]
students.head()

,student_id,subject_id,grade
1,1,0,3.0
2,1,0,3.0
3,1,0,3.0
4,1,0,3.0
5,2,0,3.0


In [28]:
# Повторная генерация всех необходимых данных в одном месте
import pandas as pd
import string
random.seed(0)
# Базовые слова для логинов
base_words = [
    "sky", "river", "stone", "leaf", "fire", "cloud", "storm", "wolf", "hawk", "tree",
    "wind", "light", "dark", "moon", "star", "snow", "rock", "grass", "wave", "sun"
]

# Генерация логинов
logins = set()
while len(logins) < 73:
    word = random.choice(base_words)
    number = random.randint(10, 99)
    login = f"{word}_{number}"
    logins.add(login)
logins = list(logins)

# Генерация паролей
characters = string.ascii_letters + string.digits
passwords = [''.join(random.choices(characters, k=8)) for _ in range(73)]

# Генерация телефонов
phones = [f"+7{random.randint(9000000000, 9999999999)}" for _ in range(73)]

# Генерация email (на основе логинов)
domains = ["example.com", "mail.com", "test.org", "demo.net"]
emails = [f"{login}@{random.choice(domains)}" for login in logins]

# Объединение в DataFrame
accounts_df = pd.DataFrame({
    "login": logins,
    "password": passwords,
    "phone": phones,
    "email": emails
})

accounts_df.head()


,login,password,phone,email
0,river_80,ZhfZhLm8,+79093017884,river_80@mail.com
1,rock_72,SDTdQHMZ,+79756906071,rock_72@test.org
2,sky_88,eesBdCKR,+79932913624,sky_88@mail.com
3,grass_97,yq4D3JRm,+79549883210,grass_97@mail.com
4,hawk_70,VWjkGh5P,+79806908518,hawk_70@test.org


In [29]:
site_user = accounts_df.copy()
site_user['user_id'] = site_user.index+1
site_user['type'] = 'student'
site_user = site_user[['user_id', 'login', 'password', 'type', 'phone', 'email']]
site_user.head()

,user_id,login,password,type,phone,email
0,1,river_80,ZhfZhLm8,student,+79093017884,river_80@mail.com
1,2,rock_72,SDTdQHMZ,student,+79756906071,rock_72@test.org
2,3,sky_88,eesBdCKR,student,+79932913624,sky_88@mail.com
3,4,grass_97,yq4D3JRm,student,+79549883210,grass_97@mail.com
4,5,hawk_70,VWjkGh5P,student,+79806908518,hawk_70@test.org


In [30]:
user_student = pd.DataFrame(site_user['user_id'].reset_index(drop=True), columns=['user_id'])
user_student['student_id'] = student_info['student_id'].reset_index(drop=True)
user_student

,user_id,student_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
68,69,69
69,70,70
70,71,71
71,72,72


In [31]:
# Повторная генерация всех необходимых данных в одном месте
import pandas as pd
import string
random.seed(0)
# Базовые слова для логинов
base_words = [
    "sky", "river", "stone", "leaf", "fire", "cloud", "storm", "wolf", "hawk", "tree",
    "wind", "light", "dark", "moon", "star", "snow", "rock", "grass", "wave", "sun"
]
n = 5
# Генерация логинов
logins = set()
while len(logins) < n:
    word = random.choice(base_words)
    number = random.randint(10, 99)
    login = f"{word}_{number}"
    logins.add(login)
logins = list(logins)

# Генерация паролей
characters = string.ascii_letters + string.digits
passwords = [''.join(random.choices(characters, k=8)) for _ in range(n)]

# Генерация телефонов
phones = [f"+7{random.randint(9000000000, 9999999999)}" for _ in range(n)]

# Генерация email (на основе логинов)
domains = ["example.com", "mail.com", "test.org", "demo.net"]
emails = [f"{login}@{random.choice(domains)}" for login in logins]

# Объединение в DataFrame
accounts_df = pd.DataFrame({
    "login": logins,
    "password": passwords,
    "phone": phones,
    "email": emails
})

accounts_df.head()


,login,password,phone,email
0,rock_72,K4FrUMp4,+79153002188,rock_72@example.com
1,river_43,8Y3tT3QD,+79862407391,river_43@test.org
2,dark_63,gAL47D1q,+79583031024,dark_63@demo.net
3,snow_55,XIaSyZPa,+79481003665,snow_55@example.com
4,dark_48,E1pu1lJo,+79097942384,dark_48@test.org


In [32]:
teachers = accounts_df.copy()
teachers['user_id'] = teachers.index+74
teachers['type'] = 'teacher'
teachers = teachers[['user_id', 'login', 'password', 'type', 'phone', 'email']]
teachers.head()

,user_id,login,password,type,phone,email
0,74,rock_72,K4FrUMp4,teacher,+79153002188,rock_72@example.com
1,75,river_43,8Y3tT3QD,teacher,+79862407391,river_43@test.org
2,76,dark_63,gAL47D1q,teacher,+79583031024,dark_63@demo.net
3,77,snow_55,XIaSyZPa,teacher,+79481003665,snow_55@example.com
4,78,dark_48,E1pu1lJo,teacher,+79097942384,dark_48@test.org


In [33]:
site_user = pd.concat([site_user,teachers])
site_user

,user_id,login,password,type,phone,email
0,1,river_80,ZhfZhLm8,student,+79093017884,river_80@mail.com
1,2,rock_72,SDTdQHMZ,student,+79756906071,rock_72@test.org
2,3,sky_88,eesBdCKR,student,+79932913624,sky_88@mail.com
3,4,grass_97,yq4D3JRm,student,+79549883210,grass_97@mail.com
4,5,hawk_70,VWjkGh5P,student,+79806908518,hawk_70@test.org
...,...,...,...,...,...,...
0,74,rock_72,K4FrUMp4,teacher,+79153002188,rock_72@example.com
1,75,river_43,8Y3tT3QD,teacher,+79862407391,river_43@test.org
2,76,dark_63,gAL47D1q,teacher,+79583031024,dark_63@demo.net
3,77,snow_55,XIaSyZPa,teacher,+79481003665,snow_55@example.com


In [34]:
# Повторная генерация всех необходимых данных в одном месте
import pandas as pd
import string
random.seed(1)
# Базовые слова для логинов
base_words = [
    "sky", "river", "stone", "leaf", "fire", "cloud", "storm", "wolf", "hawk", "tree",
    "wind", "light", "dark", "moon", "star", "snow", "rock", "grass", "wave", "sun"
]
n = 5
# Генерация логинов
logins = set()
while len(logins) < n:
    word = random.choice(base_words)
    number = random.randint(10, 99)
    login = f"{word}_{number}"
    logins.add(login)
logins = list(logins)

# Генерация паролей
characters = string.ascii_letters + string.digits
passwords = [''.join(random.choices(characters, k=8)) for _ in range(n)]

# Генерация телефонов
phones = [f"+7{random.randint(9000000000, 9999999999)}" for _ in range(n)]

# Генерация email (на основе логинов)
domains = ["example.com", "mail.com", "test.org", "demo.net"]
emails = [f"{login}@{random.choice(domains)}" for login in logins]

# Объединение в DataFrame
accounts_df = pd.DataFrame({
    "login": logins,
    "password": passwords,
    "phone": phones,
    "email": emails
})

accounts_df.head()


,login,password,phone,email
0,fire_82,fbZAVaBS,+79891244035,fire_82@test.org
1,dark_36,o63bbH6x,+79977303767,dark_36@demo.net
2,star_70,nAbnBEoo,+79719735122,star_70@demo.net
3,stone_42,nCrbZINl,+79203849597,stone_42@example.com
4,leaf_73,91huSS6A,+79325739463,leaf_73@demo.net


In [35]:
parents = accounts_df.copy()
parents['user_id'] = parents.index+79
parents['type'] = 'parent'
parents = parents[['user_id', 'login', 'password', 'type', 'phone', 'email']]
parents.head()
site_user = pd.concat([site_user,parents])
site_user

,user_id,login,password,type,phone,email
0,1,river_80,ZhfZhLm8,student,+79093017884,river_80@mail.com
1,2,rock_72,SDTdQHMZ,student,+79756906071,rock_72@test.org
2,3,sky_88,eesBdCKR,student,+79932913624,sky_88@mail.com
3,4,grass_97,yq4D3JRm,student,+79549883210,grass_97@mail.com
4,5,hawk_70,VWjkGh5P,student,+79806908518,hawk_70@test.org
...,...,...,...,...,...,...
0,79,fire_82,fbZAVaBS,parent,+79891244035,fire_82@test.org
1,80,dark_36,o63bbH6x,parent,+79977303767,dark_36@demo.net
2,81,star_70,nAbnBEoo,parent,+79719735122,star_70@demo.net
3,82,stone_42,nCrbZINl,parent,+79203849597,stone_42@example.com


In [36]:
columns = ['user_id', 'student_id']
parents_user = pd.DataFrame([
    [79, 1], [80, 2], [81, 3], [82, 4], [83, 5]
], columns=columns)
parents_user

,user_id,student_id
0,79,1
1,80,2
2,81,3
3,82,4
4,83,5


In [37]:
user_student = pd.concat([user_student, parents_user])
user_student

,user_id,student_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
0,79,1
1,80,2
2,81,3
3,82,4


In [38]:
columns = ['user_id', 'class_id']
teacher_class = pd.DataFrame([
    [74, 1], [74, 2], [74, 3], [74, 6], [75, 2], [76, 3], [77, 4], [78, 5]
], columns=columns)
teacher_class

,user_id,class_id
0,74,1
1,74,2
2,74,3
3,74,6
4,75,2
5,76,3
6,77,4
7,78,5


In [39]:
columns = ['user_id', 'subject_id']
teacher_subject = pd.DataFrame([
    [74, 0], [75, 21], [75, 8], [75, 12], [76, 13], [77, 9], [78, 18]
], columns=columns)
teacher_subject

,user_id,subject_id
0,74,0
1,75,21
2,75,8
3,75,12
4,76,13
5,77,9
6,78,18


In [40]:
subjects.columns = ['subject_id', 'name']
subjects.head()

,subject_id,name
0,0,Алг.
1,1,Англ.яз
2,2,Биол.
3,3,ГД
4,4,Геогр.


In [41]:
students.drop_duplicates(subset=['student_id', 'subject_id'],inplace=True)

In [42]:
student_info.to_csv('student_info.csv')
subjects.to_csv('subjects.csv')
classes.to_csv('classes.csv')
teacher_subject.to_csv('teacher_subject.csv')
site_user.to_csv('site_user.csv')
teacher_class.to_csv('teacher_class.csv')
user_student.to_csv('user_student.csv')
students.to_csv('students.csv')